### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 25 ноября 2019, 15:00   
**Штраф за опоздание:** -2 балла после 15:00 25 ноября, -4 балла после 15:00 2 декабря, -6 баллов после 15:00 9 декабря  -8 баллов после 15:00 16 декабря.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0919, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на [wine](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html) и [Speed Dating Data](https://cloud.mail.ru/public/8nHV/p6J7wY1y1)

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

from math import log
from math import sqrt
#from random import randint
from functools import reduce
#import time

%matplotlib inline
#%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):#работаю с матрицами NxKxF N - число объектов
        l_s = l_s.astype('float')        # K - число классов, F - чилсо признаков. r_c - общее количество объектов справа 
        r_s = r_s.astype('float')        # r_s количество объектов справа по классам. l_s такая матрица, что 
                                         # l_s - r_s = количество объектов слева по классам. l_c такая матрица, что
                                         # l_c - r_c = общее количество объектов слева
        return ((r_s-l_s*(r_c/l_c))**2).sum(axis=1)/((l_c-r_c)*r_c).reshape(-1,1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return reduce(lambda ans,x:ans+(x[0]*np.log(x[0]/x[1]+0.000000000001)).sum(axis=1),
                      [[l_s-r_s,l_c-r_c],[r_s,r_c],[-l_s,-l_c]],0)/l_c

    def __misclass(self, l_c, l_s, r_c, r_s):
        return ((l_s-r_s)/(l_c-r_c)).max(axis=1)+(r_s/r_c).max(axis=1)-(l_s/l_c).max(axis=1)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(sqrt(n_feature))]
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = np.arange(n_feature)
        np.random.shuffle(feature_ids)
        return feature_ids[:int(log(n_feature,2))]

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        y_un=np.unique(y)
        LS=(y[...,np.newaxis]==y_un[np.newaxis]).sum(axis=0)
        if LS.max()==y.shape[0]:
            return [-1,-1]
        
        best_gain=0
        best_feature=best_th=-1
        
        rc=np.arange(1,y.shape[0])
        
        Y=y.shape[0]
        
        feature_ids=self.get_feature_ids(x.shape[1])
        sorted_ids=x[:,feature_ids].argsort(axis=0)
        rs=(y[sorted_ids][:,np.newaxis]==y_un[np.newaxis,:,np.newaxis]).astype(int)
        for i in range(1,y.shape[0]):
                rs[i]+=rs[i-1]
        R=rs[-1]
        rs=rs[:-1]
        gain=self.G_function(Y,R[np.newaxis],rc[...,np.newaxis,np.newaxis],rs)
        r=np.take_along_axis(x,sorted_ids,axis=0)
        gain*=r[:-1]!=r[1:]
        best_idx=gain.argmax()
        if gain.ravel()[best_idx]>0:
            best_feature=feature_ids[best_idx%gain.shape[1]]
            best_th=x[sorted_ids.ravel()[best_idx],best_feature]
            self.feature_importances_[best_feature]+=y.shape[0]/self.num_samples*gain.ravel()[best_idx]
    
        return best_feature,best_th
            

    def __fit_node(self, x, y, node_id, depth):
        if (self.max_depth is not None and depth>=self.max_depth) or y.shape[0]<self.min_samples_split:
            counts=(y[...,np.newaxis]==self.num_class[np.newaxis]).sum(axis=0) 
            self.tree[node_id]=[self.__class__.LEAF_TYPE,
                           self.num_class[counts.argmax()],counts/counts.sum()]
            return None
        if self.sufficient_share<1:
            counts=(y[...,np.newaxis]==self.num_class[np.newaxis]).sum(axis=0)
            if counts.max()/counts.sum()>=self.sufficient_share:
                self.tree[node_id]=[self.__class__.LEAF_TYPE,
                           self.num_class[counts.argmax()],counts/counts.sum()]
                return None

        feature_id,th=self.__find_threshold(x,y)

        if feature_id==-1:
            counts=(y[...,np.newaxis]==self.num_class[np.newaxis]).sum(axis=0) 
            self.tree[node_id]=[self.__class__.LEAF_TYPE,
                           self.num_class[counts.argmax()],counts/counts.sum()]
        else:
            self.tree[node_id]=[self.__class__.NON_LEAF_TYPE,feature_id,th]
            xl,xr,yl,yr=self.__div_samples(x, y, feature_id, th)

            self.__fit_node(xl,yl,node_id*2+1,depth+1)
            self.__fit_node(xr,yr,node_id*2+2,depth+1)

        
                
    
    def fit(self, x, y):
        self.num_class = np.unique(y)
        self.feature_importances_=np.zeros(x.shape[1])
        self.num_samples=x.shape[0]
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

Wall time: 1.95 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

Wall time: 10.8 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9487179487179486

## Подготовка данных Speed Dating Data 

In [9]:
df=pd.read_csv('data/speed-dating-experiment/Speed Dating Data.csv',encoding='latin1')
df = df.iloc[:, :97]
df.shape

(8378, 97)

In [10]:
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
df.loc[:, 'income'] = df.loc[:, 'income'].str.replace(',', '').astype(np.float)
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)

In [11]:
df=df.drop(['id','idg','condtn','round','position', 'positin1',
          'order','partner','age_o', 'race_o', 'pf_o_att',
          'pf_o_sin', 'pf_o_int',
          'pf_o_fun', 'pf_o_amb', 'pf_o_sha',
          'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o',
          'amb_o', 'shar_o', 'like_o', 'prob_o','met_o',
         'field','undergra','from', 'zipcode','career','expnum'], axis=1)

In [12]:
df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(19)
df.loc[:, 'race'] = df.loc[:, 'race'].fillna(7)
df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(18)
df.loc[:, 'goal'] = df.loc[:, 'goal'].fillna(7)
l=['field_cd','race','career_c','goal']

for feature in l:
    un=df[feature].unique()
    a=(df[feature].values[...,np.newaxis]==un[np.newaxis]).astype(np.float)
    for j,i in enumerate(un):
        df[feature+'_'+str(i)]=a[:,j]
df=df.drop(['field_cd','race','career_c','goal'],axis=1)

In [13]:
df = df.drop(['sports','tvsports','exercise','dining','museums','art','hiking','gaming',
       'clubbing','reading','tv','theater','movies','concerts','music','shopping','yoga'], axis=1)

In [14]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].sum(axis=1)
df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']] = \
(df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100

In [15]:
df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].sum(axis=1)
df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']] = \
(df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1']].T/df.loc[:, 'temp_totalsum'].T).T * 100
df = df.drop(['temp_totalsum'], axis=1)

In [16]:
for i in [4, 5]:
    feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i), 
            'intel{}_1'.format(i), 'fun{}_1'.format(i), 
            'amb{}_1'.format(i), 'shar{}_1'.format(i)]
    
    if i != 4:
        feat.remove('shar{}_1'.format(i))
    
    df = df.drop(feat, axis=1)

In [17]:
df = df.drop(['wave'], axis=1)
df.shape

(8378, 82)

In [18]:
df.loc[:, 'tuition'] = df.loc[:, 'tuition'].fillna(df.loc[:, 'tuition'].mean())
df.loc[:, 'mn_sat'] = df.loc[:, 'mn_sat'].fillna(df.loc[:, 'mn_sat'].mean())
df.loc[:, 'income'] = df.loc[:, 'income'].fillna(df.loc[:, 'income'].mean())

In [19]:
df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                                 .drop(['gender'], axis=1)\
                                 .dropna()
df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                                   .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                                   .dropna()
        
df_female.columns = df_female.columns + '_f'

In [20]:
ans=pd.merge(df_male,df_female,left_on=['pid'],right_on=['iid_f'],how='inner')
ans = ans.drop(['iid','pid','pid_f','iid_f'], axis=1)
ans.shape

(3999, 155)

In [21]:
ans['age_dif']=ans.age-ans.age_f

In [22]:
y=ans['match'].values
X=ans.drop(['match'],axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [23]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [24]:
%time clf.fit(X_train, y_train)

Wall time: 108 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [25]:
%time my_clf.fit(X_train, y_train)

Wall time: 1 s


## Проверка качества работы на Speed Dating Data

In [26]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5437565318947475

In [27]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5401858415557046

## Задание 4

In [28]:
a=clf.feature_importances_.argsort()
print('Decision Tree')
np.array(ans.drop(['match'],axis=1).columns.tolist())[a[::-1][:10]]

Decision Tree


array(['int_corr', 'age_dif', 'amb2_1_f', 'fun1_1_f', 'amb1_1', 'sinc1_1',
       'attr3_1_f', 'amb1_1_f', 'sinc2_1_f', 'attr3_1'], dtype='<U15')

In [29]:
a=my_clf.feature_importances_.argsort()
print("My Decision Tree")
np.array(ans.drop(['match'],axis=1).columns.tolist())[a[::-1][:10]]

My Decision Tree


array(['int_corr', 'age', 'age_dif', 'sinc1_1_f', 'fun1_1_f', 'amb2_1_f',
       'income_f', 'attr3_1', 'attr1_1_f', 'amb1_1'], dtype='<U15')

## Задание 5

In [30]:
rfc=RandomForestClassifier(random_state=42)

In [32]:
param_grid = { 
    'n_estimators': [500,700,900],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,10,None],
    'criterion' :['gini', 'entropy']
}

In [33]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'

In [34]:
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'n_estimators': 500}

In [35]:
rfc=RandomForestClassifier(random_state=42,criterion='gini',max_depth=None,max_features='auto',n_estimators=500)

In [36]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [37]:
f1_score(y_pred=rfc.predict(X_test), y_true=y_test, average='macro')

0.49767190247106785